In [ ]:
%matplotlib notebook
%precision 3

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [9.5, 7]
import datetime
import pandas



In [ ]:
prefix="/home/ahu/git/gazproject/"

In [ ]:
def cleanUp(v):
    v["timestamp"]=pandas.to_datetime(v["periodFrom"])
    v["timestamp"]=v["timestamp"].dt.strftime("%Y-%m-%d 06:00:00")
    v["timestamp"]=pandas.to_datetime(v["timestamp"])
    v.sort_values(["timestamp"], inplace=True)
    v.set_index("timestamp", inplace=True)
    
uzhgorod = pandas.read_json(prefix+"uzhgorod-data.json")
nordstreamF = pandas.read_json(prefix+"nordstream-fluxsys-data.json")
nordstreamO = pandas.read_json(prefix+"nordstream-opal-data.json")
yamalK = pandas.read_json(prefix+"yamal-kondratki-data.json")
yamalW = pandas.read_json(prefix+"yamal-wysokoje-data.json")
hermanowice = pandas.read_json(prefix+"hermanowice-data.json")

cleanUp(uzhgorod)
cleanUp(nordstreamF)
cleanUp(nordstreamO)
cleanUp(yamalK)
cleanUp(yamalW)
cleanUp(hermanowice)


In [ ]:
plt.figure()
plt.plot(uzhgorod.value/24/1000000, label="Uzhgorod")
plt.plot(nordstreamF.value/24/1000000, label="Nordstream Greifswald Fluxsys")
plt.plot(nordstreamO.value/24/1000000, label="Nordstream Greifswald Opal")
plt.plot(yamalK.value/24/1000000, label="Yamal Kondratki")
plt.plot(yamalW.value/24/1000000, label="Yamal Wysokoje")
plt.plot(hermanowice.value/24/1000000, label="Hermanowice")

plt.plot((uzhgorod.value+yamalK.value+yamalW.value+nordstreamF.value+nordstreamO.value+hermanowice.value)/24000000, label="Sum")

plt.ylabel("GW")
plt.legend()
plt.title("Russian natural gas flow to Europe")
plt.xticks(rotation=25)

plt.grid()

In [ ]:
plt.figure()
#yamalK=yamalK[:"2022-03-12"]
#yamalW=yamalW[:"2022-03-12"]
plt.stackplot(uzhgorod.index,
              uzhgorod.value/24000000, nordstreamF.value/24000000, nordstreamO.value/24000000,
              yamalK.value/24000000,
              yamalW.value/24000000, hermanowice.value/24000000,

             labels=["Uzhgorod", "Nordstream Fluxsys", "Nordstream OPAL", "Yamal Kondratki", "Yamal Wysokoje", "Hermanowice"])
plt.legend(loc=2)
plt.ylabel("GW")
plt.title("Russian natural gas flow to Europe")
plt.xticks(rotation=25)

plt.grid()